## Pandas - operacje na dacie i czasie

#### Autor: Marian Witkowski marian.witkowski[at]gmail.com

#### Niniejszy materiał może być używany w celach dydaktyczych pod warunkem poinformowania o autorze.

Przykład użycia najczęstszych operacji i technik manulacji na obiektach przechowujących informacje o dacie/czasie w Pandas:
- ładowanie danych z CSV
- konwersja danych object do datetime64[ns]
- dostęp do właściwości .dt
- odległość pomiędzy datami
- wybieranie danych z wykorzystaniem formatu daty ISO8601
- agregowanie danych
- rolowanie danych
- generowanie czasokresów
- znajdowanie początku/końca okresu czas
- łączenie/merge'owanie obiektów DataFrame z wykorzystaniem czasu i zadanej tolerancji dla offsetu wartości

<img src='http://51.91.120.89/itm/p.png' border=0 />

In [116]:
import datetime
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

## Ładowanie danych do DataFrame

In [117]:
df = pd.read_csv("https://stooq.pl/q/d/l/?s=tsla.us&d1=20150101&d2=20221130&i=d")

### Notowania firmy TESLA od 2015-01-01 do 2022-11-30

In [118]:
df

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,2015-01-02,14.8580,14.8833,14.2173,14.6207,71461545
1,2015-01-05,14.3000,14.4333,13.8107,14.0060,80488110
2,2015-01-06,14.0040,14.2800,13.6140,14.0853,93924045
3,2015-01-07,14.2233,14.3187,13.9853,14.0633,44514750
4,2015-01-08,14.1873,14.2533,14.0007,14.0413,51634560
...,...,...,...,...,...,...
1987,2022-11-22,168.6300,170.9200,166.1850,169.9100,78452327
1988,2022-11-23,173.5700,183.6200,172.5000,183.2000,109536709
1989,2022-11-25,185.0600,185.2000,180.6300,182.8600,50672739
1990,2022-11-28,179.9600,188.5000,179.0000,182.9200,93038148


### Zakres min. i maks. dla dat

In [119]:
pd.Timestamp.min, pd.Timestamp.max

(Timestamp('1677-09-21 00:12:43.145224193'),
 Timestamp('2262-04-11 23:47:16.854775807'))

### Konwersja kolumny "Data" do dedykowane typu datetime64[ns]

In [120]:
df.Data = pd.to_datetime(df.Data, format="%Y-%m-%d", errors="coerce")
df.dtypes

Data          datetime64[ns]
Otwarcie             float64
Najwyzszy            float64
Najnizszy            float64
Zamkniecie           float64
Wolumen                int64
dtype: object

### Obiekty akcesora dla właściwości dt

In [121]:
print([prop for prop in dir(df.Data.dt) if prop[0]!='_'])

['ceil', 'date', 'day', 'day_name', 'day_of_week', 'day_of_year', 'dayofweek', 'dayofyear', 'days_in_month', 'daysinmonth', 'floor', 'freq', 'hour', 'is_leap_year', 'is_month_end', 'is_month_start', 'is_quarter_end', 'is_quarter_start', 'is_year_end', 'is_year_start', 'isocalendar', 'microsecond', 'minute', 'month', 'month_name', 'nanosecond', 'normalize', 'quarter', 'round', 'second', 'strftime', 'time', 'timetz', 'to_period', 'to_pydatetime', 'tz', 'tz_convert', 'tz_localize', 'week', 'weekday', 'weekofyear', 'year']


### Pobranie informacji o roku, numerze tygodnia, miesiącu, dniu tygodnia, przestępności roku z daty w kolumnie "Data"

In [122]:
df["year"] = df.Data.dt.year
df["week_number"] = df.Data.dt.weekofyear
df["month"] = df.Data.dt.month
df["day_of_week"] = df.Data.dt.day_of_week
df["is_leap_year"] = df.Data.dt.is_leap_year
df.sample(n=10)

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,year,week_number,month,day_of_week,is_leap_year
863,2018-06-07,21.0767,22.0000,20.9053,21.0727,215179065,2018,23,6,3,False
650,2017-08-02,21.2627,21.8080,20.7480,21.7260,181369545,2017,31,8,2,False
1396,2020-07-21,109.3300,111.6670,103.8670,104.5570,242359200,2020,30,7,1,True
1788,2022-02-08,301.8430,308.7630,298.2670,307.3330,50729013,2022,6,2,1,False
1290,2020-02-19,61.5667,62.9867,60.0667,61.1600,381344370,2020,8,2,2,True
1112,2019-06-05,13.2453,13.4187,12.7900,13.1060,202661340,2019,23,6,2,False
1107,2019-05-29,12.4733,12.8260,12.3360,12.6573,179529570,2019,22,5,2,False
1756,2021-12-22,321.8870,338.5530,319.0170,336.2900,93634086,2021,51,12,2,False
1199,2019-10-08,15.7247,16.2627,15.6333,16.0033,130535070,2019,41,10,1,False
55,2015-03-24,13.4387,13.5860,13.3167,13.4480,54747000,2015,13,3,1,False


### Obliczanie wartości pomiędzy datami

In [123]:
df1 = df.sample(n=10, random_state=0, ignore_index=True)[["Data"]]
df2 = df.sample(n=10, random_state=42, ignore_index=True)[["Data"]]
df_tmp = pd.concat([df1,df2], axis=1)
df_tmp.columns = ['Data1', 'Data2']
df_tmp

,Data1,Data2
0,2022-03-03,2018-07-12
1,2020-03-19,2021-08-23
2,2018-10-24,2016-08-24
3,2015-06-08,2019-04-18
4,2021-03-11,2019-05-21
5,2017-10-18,2018-12-19
6,2022-09-27,2022-08-29
7,2022-05-23,2022-10-24
8,2020-12-22,2018-01-19
9,2015-01-29,2021-04-19


In [124]:
time_delta = (df_tmp.Data1 - df_tmp.Data2).dt.total_seconds()
df_tmp["minutes_diff"] = time_delta / 60
df_tmp["hours_diff"] = time_delta / 3600
df_tmp["days_diff"] = time_delta / (3600*24)
df_tmp

,Data1,Data2,minutes_diff,hours_diff,days_diff
0,2022-03-03,2018-07-12,1915200.0,31920.0,1330.0
1,2020-03-19,2021-08-23,-751680.0,-12528.0,-522.0
2,2018-10-24,2016-08-24,1139040.0,18984.0,791.0
3,2015-06-08,2019-04-18,-2030400.0,-33840.0,-1410.0
4,2021-03-11,2019-05-21,950400.0,15840.0,660.0
5,2017-10-18,2018-12-19,-614880.0,-10248.0,-427.0
6,2022-09-27,2022-08-29,41760.0,696.0,29.0
7,2022-05-23,2022-10-24,-221760.0,-3696.0,-154.0
8,2020-12-22,2018-01-19,1537920.0,25632.0,1068.0
9,2015-01-29,2021-04-19,-3271680.0,-54528.0,-2272.0


### Wybranie danych z DataFrame z wykorzystaniem obiektów datetime64[ns]

Obowiązkowe ustawienie indeksu z wykorzystaniem danych datetime64[ns]

In [125]:
df.set_index("Data", inplace=True)
df 

,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,year,week_number,month,day_of_week,is_leap_year
Data,,,,,,,,,,
2015-01-02,14.8580,14.8833,14.2173,14.6207,71461545,2015,1,1,4,False
2015-01-05,14.3000,14.4333,13.8107,14.0060,80488110,2015,2,1,0,False
2015-01-06,14.0040,14.2800,13.6140,14.0853,93924045,2015,2,1,1,False
2015-01-07,14.2233,14.3187,13.9853,14.0633,44514750,2015,2,1,2,False
2015-01-08,14.1873,14.2533,14.0007,14.0413,51634560,2015,2,1,3,False
...,...,...,...,...,...,...,...,...,...,...
2022-11-22,168.6300,170.9200,166.1850,169.9100,78452327,2022,47,11,1,False
2022-11-23,173.5700,183.6200,172.5000,183.2000,109536709,2022,47,11,2,False
2022-11-25,185.0600,185.2000,180.6300,182.8600,50672739,2022,47,11,4,False


Data podawana jest w formacie ISO8601 <a href='https://en.wikipedia.org/wiki/ISO_8601'>https://en.wikipedia.org/wiki/ISO_8601</a>

In [126]:
df["2015-01-01":"2015-01-10"] # z podaniem daty początkowej i końcowej

,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,year,week_number,month,day_of_week,is_leap_year
Data,,,,,,,,,,
2015-01-02,14.8580,14.8833,14.2173,14.6207,71461545,2015,1,1,4,False
2015-01-05,14.3000,14.4333,13.8107,14.0060,80488110,2015,2,1,0,False
2015-01-06,14.0040,14.2800,13.6140,14.0853,93924045,2015,2,1,1,False
2015-01-07,14.2233,14.3187,13.9853,14.0633,44514750,2015,2,1,2,False
2015-01-08,14.1873,14.2533,14.0007,14.0413,51634560,2015,2,1,3,False
2015-01-09,13.9280,13.9987,13.6640,13.7773,70022775,2015,2,1,4,False


In [127]:
df["2022-11-22":] # z podaniem tylko daty początkowej 

,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,year,week_number,month,day_of_week,is_leap_year
Data,,,,,,,,,,
2022-11-22,168.63,170.92,166.185,169.91,78452327,2022,47,11,1,False
2022-11-23,173.57,183.62,172.500,183.20,109536709,2022,47,11,2,False
2022-11-25,185.06,185.20,180.630,182.86,50672739,2022,47,11,4,False
2022-11-28,179.96,188.50,179.000,182.92,93038148,2022,48,11,0,False
2022-11-29,184.99,186.38,178.750,180.83,83357111,2022,48,11,1,False


In [128]:
df["2022-11"] # z podaniem daty w formacie ROK-MIESIĄC

,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,year,week_number,month,day_of_week,is_leap_year
Data,,,,,,,,,,
2022-11-01,234.050,237.3951,227.280,227.82,62688822,2022,44,11,1,False
2022-11-02,226.040,227.8699,214.820,214.98,63070293,2022,44,11,2,False
2022-11-03,211.360,221.2000,210.140,215.31,56538848,2022,44,11,3,False
2022-11-04,222.600,223.8000,203.080,207.47,98622212,2022,44,11,4,False
2022-11-07,208.650,208.9000,196.660,197.08,93916520,2022,45,11,0,False
2022-11-08,194.020,195.2000,186.750,191.30,128803404,2022,45,11,1,False
2022-11-09,190.775,195.8900,177.120,177.59,127062659,2022,45,11,2,False
2022-11-10,189.900,191.0000,180.030,190.72,132703015,2022,45,11,3,False
2022-11-11,186.000,196.5200,182.590,195.97,114403575,2022,45,11,4,False


In [129]:
df["2022-Q3"] # z podaniem daty w formacie ROK-NR_KWARTAŁU

,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,year,week_number,month,day_of_week,is_leap_year
Data,,,,,,,,,,
2022-07-01,227.000,230.230,222.120,227.263,74460444,2022,26,7,4,False
2022-07-05,223.000,233.147,216.167,233.067,84779112,2022,27,7,1,False
2022-07-06,230.780,234.563,227.187,231.733,71853630,2022,27,7,2,False
2022-07-07,233.920,245.362,232.210,244.543,81930690,2022,27,7,3,False
2022-07-08,242.333,254.980,241.161,250.763,101854086,2022,27,7,4,False
...,...,...,...,...,...,...,...,...,...,...
2022-09-26,271.830,284.090,270.310,276.010,58076913,2022,39,9,0,False
2022-09-27,283.840,288.670,277.510,282.940,61925185,2022,39,9,1,False
2022-09-28,283.080,289.000,277.570,287.810,54664809,2022,39,9,2,False


In [130]:
df["2022-Q3": ] # z podaniem daty w formacie ROK-NR_KWARTAŁU

,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,year,week_number,month,day_of_week,is_leap_year
Data,,,,,,,,,,
2022-07-01,227.000,230.230,222.120,227.263,74460444,2022,26,7,4,False
2022-07-05,223.000,233.147,216.167,233.067,84779112,2022,27,7,1,False
2022-07-06,230.780,234.563,227.187,231.733,71853630,2022,27,7,2,False
2022-07-07,233.920,245.362,232.210,244.543,81930690,2022,27,7,3,False
2022-07-08,242.333,254.980,241.161,250.763,101854086,2022,27,7,4,False
...,...,...,...,...,...,...,...,...,...,...
2022-11-22,168.630,170.920,166.185,169.910,78452327,2022,47,11,1,False
2022-11-23,173.570,183.620,172.500,183.200,109536709,2022,47,11,2,False
2022-11-25,185.060,185.200,180.630,182.860,50672739,2022,47,11,4,False


### Grupowanie i funkcje agregujące dane datetime64[ns]

Funkcja agregujące wywoływane są dla kolumny lub kolumn w połączeniem z metodą resample() 

Więcej na https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html

In [131]:
df.resample("A")["Zamkniecie"].mean() # Średnia roczna kursu zamknięcia

Data
2015-12-31     15.335553
2016-12-31     13.984354
2017-12-31     20.954214
2018-12-31     21.154007
2019-12-31     18.235271
2020-12-31     96.665705
2021-12-31    259.998163
2022-12-31    273.490306
Freq: A-DEC, Name: Zamkniecie, dtype: float64

In [132]:
df.resample("Q")["Otwarcie"].mean() # Kwartalny średni kurs otwarcia

Data
2015-03-31     13.486592
2015-06-30     15.750368
2015-09-30     16.988381
2015-12-31     14.982797
2016-03-31     13.178726
2016-06-30     15.184041
2016-09-30     14.443305
2016-12-31     13.157600
2017-03-31     16.860819
2017-06-30     21.996594
2017-09-30     23.136532
2017-12-31     21.752959
2018-03-31     22.021861
2018-06-30     20.265328
2018-09-30     20.811805
2018-12-31     21.545724
2019-03-31     20.047582
2019-06-30     15.581778
2019-09-30     15.643155
2019-12-31     21.578659
2020-03-31     41.387763
2020-06-30     54.002784
2020-09-30    117.658589
2020-12-31    170.048625
2021-03-31    251.691902
2021-06-30    217.179556
2021-09-30    235.081063
2021-12-31    334.779594
2022-03-31    311.826113
2022-06-30    275.189694
2022-09-30    279.051797
2022-12-31    209.445610
Freq: Q-DEC, Name: Otwarcie, dtype: float64

In [133]:
df.resample("14D")["Otwarcie"].agg([np.min, np.max]) # 14-dniowy minimalny i maksymalny kurs otwarcia

,amin,amax
Data,,
2015-01-02,12.3887,14.8580
2015-01-16,12.6367,13.7407
2015-01-30,12.9047,14.8000
2015-02-13,13.5260,14.3013
2015-02-27,12.5640,13.7873
...,...,...
2022-09-30,208.3000,266.1450
2022-10-14,205.8200,229.7700
2022-10-28,189.9000,234.0500


In [134]:
df.resample("6M").agg({
    "Otwarcie" : [np.min, np.max],
    "Zamkniecie" : [np.std, np.median]
})

Otwarcie           Zamkniecie           
                amin      amax        std     median
Data                                                
2015-01-31   12.3887   14.8580   0.497424   13.59500
2015-07-31   12.3900   18.6667   1.863899   15.40200
2016-01-31   12.6633   17.7740   1.244192   15.33000
2016-07-31    9.4880   17.7633   1.743037   14.59830
2017-01-31   12.1673   17.1540   1.221621   13.60200
2017-07-31   16.2793   25.7793   2.622128   20.58870
2018-01-31   19.9667   25.3500   1.341519   22.73530
2018-07-31   16.8520   24.3440   1.651425   20.69470
2019-01-31   17.0167   25.0000   2.104862   21.39135
2019-07-31   12.0733   21.3060   2.378400   17.00200
2020-01-31   14.2400   42.6667   7.303295   21.08130
2020-07-31   24.9797  111.9300  20.748980   53.57835
2021-01-31   93.0667  297.1270  55.221886  147.25300
2021-07-31  184.1830  292.3400  23.139613  222.76700
2022-01-31  223.2490  411.4700  55.069060  308.97300
2022-07-31  207.9500  378.7670  42.173904  274.32150
2023-01-31  168.6300  311.6670  45.592863  268.21000

### "Rolowanie" danych 

Agregacja danych w ramki na których wykonywane są funkcje agregujące

In [135]:
df.rolling(255).mean() # obliczenie średniej kroczącej na oknie 255-elementowym

,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen,year,week_number,month,day_of_week,is_leap_year
Data,,,,,,,,,,
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-11-22,284.773816,291.608139,276.572305,283.781698,8.056807e+07,2021.882353,26.831373,6.623529,2.027451,0.0
2022-11-23,284.008012,290.874622,275.843520,283.066953,8.075176e+07,2021.886275,26.835294,6.623529,2.023529,0.0
2022-11-25,283.297306,290.112375,275.123509,282.297698,8.069586e+07,2021.890196,26.839216,6.623529,2.023529,0.0


### Generowanie  czasokresów

Generowanie czasokresów tygodniowych zaczynających się w poniedziałek od 2022-01-01 do 2022-12-31 (daty podawane są w formacie ISO8601 lub jako obiekt pd.Timestamp()

Aliasy częstotliwości dla czasokresów dostępne na https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases

In [136]:
pd.date_range("2022-01-01", "2022-12-31", freq="W-MON")

DatetimeIndex(['2022-01-03', '2022-01-10', '2022-01-17', '2022-01-24',
               '2022-01-31', '2022-02-07', '2022-02-14', '2022-02-21',
               '2022-02-28', '2022-03-07', '2022-03-14', '2022-03-21',
               '2022-03-28', '2022-04-04', '2022-04-11', '2022-04-18',
               '2022-04-25', '2022-05-02', '2022-05-09', '2022-05-16',
               '2022-05-23', '2022-05-30', '2022-06-06', '2022-06-13',
               '2022-06-20', '2022-06-27', '2022-07-04', '2022-07-11',
               '2022-07-18', '2022-07-25', '2022-08-01', '2022-08-08',
               '2022-08-15', '2022-08-22', '2022-08-29', '2022-09-05',
               '2022-09-12', '2022-09-19', '2022-09-26', '2022-10-03',
               '2022-10-10', '2022-10-17', '2022-10-24', '2022-10-31',
               '2022-11-07', '2022-11-14', '2022-11-21', '2022-11-28',
               '2022-12-05', '2022-12-12', '2022-12-19', '2022-12-26'],
              dtype='datetime64[ns]', freq='W-MON')

Generowanie czasokresów o odstępie 2godz. 30min. 15sek.

In [137]:
pd.date_range(pd.Timestamp("2022-01-01 04:05:06"), pd.Timestamp("2022-01-01 12:05:06"), freq="2H30MIN15S")

DatetimeIndex(['2022-01-01 04:05:06', '2022-01-01 06:35:21',
               '2022-01-01 09:05:36', '2022-01-01 11:35:51'],
              dtype='datetime64[ns]', freq='9015S')

Generowanie czasokresów co kwartał

In [138]:
pd.date_range("2022-01-01", "2022-12-31", freq="Q")

DatetimeIndex(['2022-03-31', '2022-06-30', '2022-09-30', '2022-12-31'], dtype='datetime64[ns]', freq='Q-DEC')

Generowanie czasokresów co kwartał - znacznik czasu na początku czasokresu

In [139]:
pd.date_range("2022-01-01", "2022-12-31", freq="QS")

DatetimeIndex(['2022-01-01', '2022-04-01', '2022-07-01', '2022-10-01'], dtype='datetime64[ns]', freq='QS-JAN')

Generowanie wg określonej ilości znaczników o zadanej częstotliwości, począwszy od podanej daty

In [140]:
pd.date_range("2022-01-11", freq="14D", periods=10)

DatetimeIndex(['2022-01-11', '2022-01-25', '2022-02-08', '2022-02-22',
               '2022-03-08', '2022-03-22', '2022-04-05', '2022-04-19',
               '2022-05-03', '2022-05-17'],
              dtype='datetime64[ns]', freq='14D')

### Przydatne operacje na datach

Obliczenie liczby dni roboczych pomiędzy datami

In [141]:
start = datetime.date(2022, 4, 25)
end = datetime.date(2022, 5, 15)

days = np.busday_count(start, end)
print(f'Liczba dni roboczych = {days}')

Liczba dni roboczych = 15


Obliczenie liczby dni roboczych pomiędzy datami z wyłączeniem dodatkowych dni świątecznych

In [142]:
start = datetime.date(2022, 4, 25)
end = datetime.date(2022, 5, 15)

days = np.busday_count(start, end, holidays=['2022-05-01','2022-05-03'])
print(f'Liczba dni roboczych = {days}')

Liczba dni roboczych = 14


Ostatni dzień miesiąca dla podanej daty

In [143]:
pd.Timestamp("2022-11-11") + pd.offsets.MonthEnd()

Timestamp('2022-11-30 00:00:00')

Pierwszy dzień kwartału dla daty

In [144]:
pd.Timestamp("2022-11-11") - pd.offsets.QuarterBegin()

Timestamp('2022-09-01 00:00:00')

Za 2 godziny robocze od 2022-11-30 18:00 (dzień roboczy 6:00-14:00)

In [145]:
pd.Timestamp('2022-11-30 18:00') + pd.offsets.BusinessHour(2, start="06:00", end="14:00")

Timestamp('2022-12-01 08:00:00')

Konwersja do czasu wskazanej strefy czasowej z czasu lokalnego

In [146]:
ts = pd.Timestamp('2022-11-11T11:12:13.4324533')
ts.tz_localize(tz='Asia/Tokyo')

Timestamp('2022-11-11 11:12:13.432453300+0900', tz='Asia/Tokyo')

Formatowanie daty/czasu

In [147]:
ts = pd.date_range(pd.Timestamp("2022-01-01 04:05:06"), pd.Timestamp("2022-01-01 12:05:06"), freq="2H30MIN15S")
ts.strftime('%A, %Y %b %d')

Index(['Saturday, 2022 Jan 01', 'Saturday, 2022 Jan 01',
       'Saturday, 2022 Jan 01', 'Saturday, 2022 Jan 01'],
      dtype='object')

Konwersja obiektu Timedelta

In [148]:
timedelta = pd.Timedelta('7 days 20 min 15 s 35 ms')
print(f"Całkowita liczba sekund: {timedelta.total_seconds()}")
print(f"Konwersja do formatu ISO8601: {timedelta.isoformat()}")

Całkowita liczba sekund: 606015.035
Konwersja do formatu ISO8601: P7DT0H20M15.035S


### Łączenie obiektów na podstawie danych datetime64[ns] z zadanym offsetem

Wykorzystanie metody pd.merge_asof()

In [149]:
df_left = pd.DataFrame({
    "time": [pd.Timestamp("2020-03-25 13:30:00.023"),
        pd.Timestamp("2020-03-25 13:30:00.023"),
        pd.Timestamp("2020-03-25 13:30:00.030"),
        pd.Timestamp("2020-03-25 13:30:00.041"),
        pd.Timestamp("2020-03-25 13:30:00.048"),
        pd.Timestamp("2020-03-25 13:30:00.049"),
        pd.Timestamp("2020-03-25 13:30:00.072"),
        pd.Timestamp("2020-03-25 13:30:00.075")
    ],
    "ticker": ["GOOG", "MSFT", "MSFT", "MSFT", "GOOG", "AAPL", "GOOG", "MSFT"],
    "bid": [720.50, 51.95, 51.97, 51.99, 720.50, 97.99, 720.50, 52.01],
    "ask": [720.93, 51.96, 51.98, 52.00, 720.93, 98.01, 720.88, 52.03]
})

df_right = pd.DataFrame({
    "time": [
        pd.Timestamp("2020-03-25 13:30:00.023"),
        pd.Timestamp("2020-03-25 13:30:00.038"),
        pd.Timestamp("2020-03-25 13:30:00.048"),
        pd.Timestamp("2020-03-25 13:30:00.048"),
        pd.Timestamp("2020-03-25 13:30:00.048")
    ],
    "ticker": ["MSFT", "MSFT", "GOOG", "GOOG", "AAPL"],
    "price": [51.95, 51.95, 720.77, 720.92, 98.0],
    "quantity": [75, 155, 100, 100, 100]
})

df_result = pd.merge_asof(df_left, df_right, on="time",	by="ticker", tolerance=pd.Timedelta("10ms"))
df_result

,time,ticker,bid,ask,price,quantity
0,2020-03-25 13:30:00.023,GOOG,720.50,720.93,NaN,NaN
1,2020-03-25 13:30:00.023,MSFT,51.95,51.96,51.95,75.0
2,2020-03-25 13:30:00.030,MSFT,51.97,51.98,51.95,75.0
3,2020-03-25 13:30:00.041,MSFT,51.99,52.00,51.95,155.0
4,2020-03-25 13:30:00.048,GOOG,720.50,720.93,720.92,100.0
5,2020-03-25 13:30:00.049,AAPL,97.99,98.01,98.00,100.0
6,2020-03-25 13:30:00.072,GOOG,720.50,720.88,NaN,NaN
7,2020-03-25 13:30:00.075,MSFT,52.01,52.03,NaN,NaN
